20class分類を行う．

# 1. 基本設定

In [2]:
cd ..

/home/is/akiyoshi-n/my-project


# 2. 使用ライブラリ

In [3]:
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_multiclass_dataset, split_test_data, load_text_dataset
from src.my_project.train_v2 import MultiClassClassifier
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [4]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

### パラメータの設定

In [5]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 2
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 20
# 閾値
THRESH = 0.5

In [8]:
# データの読み込み
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")

In [9]:
data['labels']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [7]:
data_labels_np = np.array(data['labels'])
data_labels_np.sum(axis=0)

array([ 21,  12,  18,  46,   3,  56,  22,   1,  12,   4, 138,  28,  18,
        81,  11,  54,   5,   1,   6, 600])

In [7]:
len(data['labels'][0])

20

In [8]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name = MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [9]:
# 訓練データと評価データを辞書型で抽出
train_dataset = {
    'texts': [data['texts'][i] for i in range(900)],
    'labels': [data['labels'][i] for i in range(900)]
}
eval_dataset = {
    'texts': [data['texts'][i] for i in range(900, 1100)],
    'labels': [data['labels'][i] for i in range(900, 1100)]
}

In [10]:
trainer = Classifier_model.train_model(train_dataset, eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='21classMultiClassification', run_name='test')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.643300,0.565214,0.060000,0.045160
2,0.477800,0.353875,0.640000,0.039024
3,0.304400,0.227531,0.640000,0.039024
4,0.219600,0.177154,0.640000,0.039024
5,0.186600,0.156126,0.640000,0.039024
6,0.170200,0.143990,0.640000,0.039024
7,0.161600,0.136145,0.640000,0.039144
8,0.155000,0.130443,0.640000,0.039144
9,0.150300,0.126370,0.570000,0.038000
10,0.146500,0.122444,0.535000,0.037943


KeyboardInterrupt: 

### Cross Validation

In [6]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = MultiClassClassifier(model_name=MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [7]:
# testデータと訓練に使用するデータに分割
dataset, test_data, train_index, test_index = split_test_data(data=data, test_size=0.1, SEED=SEED)

In [8]:
result = Classifier_model.cross_validation(dataset, test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='21Classification_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.701800,0.635814,0.000000,0.075390
2,0.529200,0.395461,0.115578,0.015287
3,0.320600,0.241663,0.522613,0.035495
4,0.224700,0.195860,0.532663,0.034983
5,0.191500,0.174755,0.542714,0.035179
6,0.174500,0.162791,0.542714,0.035294
7,0.163800,0.154702,0.537688,0.035197
8,0.155600,0.149155,0.386935,0.033921
9,0.149800,0.144756,0.316583,0.032143
10,0.146000,0.140773,0.326633,0.032828


...

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.679600,0.610017,0.000000,0.070644
2,0.503300,0.378212,0.477157,0.035606
3,0.313900,0.245269,0.543147,0.035430
4,0.227400,0.199174,0.548223,0.035762
5,0.192600,0.177438,0.532995,0.036207
6,0.174700,0.164391,0.548223,0.036000
7,0.163300,0.156126,0.548223,0.036120
8,0.155100,0.149528,0.482234,0.038153
9,0.148800,0.144617,0.314721,0.032632
10,0.143200,0.140397,0.492386,0.038189


Map:   0%|          | 0/110 [00:00<?, ? examples/s]

{'eval_loss': 0.13433468341827393, 'eval_accuracy': 0.39090909090909093, 'eval_f1': 0.04710953346855984, 'eval_runtime': 0.469, 'eval_samples_per_second': 234.55, 'eval_steps_per_second': 6.397, 'epoch': 18.0}


In [9]:
result

[{'eval_loss': 0.13916701078414917,
  'eval_accuracy': 0.37272727272727274,
  'eval_f1': 0.035964912280701755,
  'eval_runtime': 0.4657,
  'eval_samples_per_second': 236.22,
  'eval_steps_per_second': 6.442,
  'epoch': 15.0},
 {'eval_loss': 0.12666897475719452,
  'eval_accuracy': 0.4727272727272727,
  'eval_f1': 0.045897435897435904,
  'eval_runtime': 0.4689,
  'eval_samples_per_second': 234.584,
  'eval_steps_per_second': 6.398,
  'epoch': 20.0},
 {'eval_loss': 0.13243137300014496,
  'eval_accuracy': 0.44545454545454544,
  'eval_f1': 0.04345930232558139,
  'eval_runtime': 0.4693,
  'eval_samples_per_second': 234.384,
  'eval_steps_per_second': 6.392,
  'epoch': 19.0},
 {'eval_loss': 0.13473385572433472,
  'eval_accuracy': 0.45454545454545453,
  'eval_f1': 0.038167938931297704,
  'eval_runtime': 0.4724,
  'eval_samples_per_second': 232.855,
  'eval_steps_per_second': 6.351,
  'epoch': 18.0},
 {'eval_loss': 0.13433468341827393,
  'eval_accuracy': 0.39090909090909093,
  'eval_f1': 0.0471

In [10]:
average_accuracy = sum(d['eval_accuracy'] for d in result)/len(result)
average_f1 = sum(d['eval_f1'] for d in result)/len(result)
print("Average accuracy:", average_accuracy)
print("Average f1:", average_f1)

Average accuracy: 0.42727272727272736
Average f1: 0.042119824580715316
